# <font color='blue'>Part 3: CRF model train and test</font>

---

We have ready-to-use data, vectorized and tagged.<br>
We present training CRF model with several paramaters, and it's results.

TBD: add tested parameter list.

## Fetch vectorized data
below we get the preprocessed data from project data folder

In [1]:
import math
import os
import sys
sys.path.append('..\GenerationPipe')
import NLP_HEBPUNCT_GP_generator as gen

# fetch vectorised data
fileName = "vectorized_data.txt"
scriptPath = os.path.dirname(os.path.realpath('__file__'))
scriptPathPrev = os.path.split(scriptPath)[0]
relFilePath_data = "\\data\\" + fileName
dataFilePath = scriptPathPrev + relFilePath_data

print("reading vectorized data from: %s" % dataFilePath)
if (os.path.isfile(dataFilePath)):
    dataFile = open(dataFilePath, "r+", encoding='utf-8')
    print("vectorized data file read OK")
    
    data = eval(dataFile.read())
    # print(data[0])
else:
    print("print: file not found in %s" % dataFilePath)

reading vectorized data from: C:\Users\moshe\Desktop\CS_courses\2018_2019\NLP\FinalProject\repository\data\vectorized_data.txt
vectorized data file read OK


## set training and test sets

In [2]:
trainRatio   = 0.9
dataSetSize  = len(data)
trainSetSize = math.ceil(dataSetSize*trainRatio)
testSetSize  = dataSetSize - trainSetSize

print("data set size       : %5d" %dataSetSize)
print("training set size   : %5d" %trainSetSize)
print("test set size       : %5d" %testSetSize)

train = data[:trainSetSize]
test  = data[trainSetSize:]

x_train = [x[0] for x in train]
y_train = [x[1] for x in train]

x_test = [x[0] for x in test]
y_test = [x[1] for x in test]


# print("-----------------------------------------------------")
# print(x_train[0])
# print("-----------------------------------------------------")
# print(y_train[0])

data set size       :  7528
training set size   :  6776
test set size       :   752


## CRF Suite - training

In [3]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)

print("fit done")

fit done


In [4]:
# TBD: remove
# y_pred = crf.predict(x_test)
# print("---------------------------------------------------")
# for i in range(len(y_test[0])):
#     if (y_pred[0][i] != y_test[0][i]):        res = "FAIL"
#     else:                                     res = "OK"
#     print("%3d - truth: %4s prediction: %4s %4s" %(i, y_test[0][i], y_pred[0][i], res))

## CRF Suite - results

In [5]:
from sklearn.metrics import classification_report

y_pred = crf.predict(x_test)

labels = list(crf.classes_)
labels.remove('none')
labels.remove('!')
labels.remove('?')

metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
print(metrics.flat_classification_report(y_pred, y_test, labels))

             precision    recall  f1-score   support

          ,       0.01      0.37      0.02       165
          .       0.19      0.96      0.32       759

avg / total       0.16      0.85      0.26       924



## CRF Suite - hyper parameters

In [7]:
from sklearn.grid_search import RandomizedSearchCV

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)

rs.fit(x_train, y_train)

# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

C:\Users\moshe\Anaconda3\lib\site-packages\sklearn\grid_search.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, Sized
C:\Users\moshe\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\moshe\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 34.3min finished


best params: {'c1': 0.2037329577368321, 'c2': 0.04593907323483032}
best CV score: 0.14425496762360251
model size: 0.33M


---

X validation results:

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.9min<br>
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 34.3min finished<br>

best params: {'c1': 0.2037329577368321, 'c2': 0.04593907323483032}<br>
best CV score: 0.14425496762360251<br>
model size: 0.33M<br>

In [ ]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.2037329577368321,
    c2=0.04593907323483032,
    max_iterations=200,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)

print("fit done")

In [ ]:
y_pred = crf.predict(x_test)

labels = list(crf.classes_)
labels.remove('none')
labels.remove('!')
labels.remove('?')

metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
print(metrics.flat_classification_report(y_pred, y_test, labels))